In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# df_short = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_17-41-16_MacdStrategyShort.json')
df_long = pd.read_json('/allah/freqtrade/json_dict/2023-09-06_18-00-38_MacdStrategyLong.json')


In [2]:
""" 30 Volume Version """
import numpy as np

lag_features = ['volume']  # List of features to create lag columns for
lag_columns = []  # List to store the names of generated lag columns

for feature in lag_features:
    for i in range(1, 21):
        df_long[f'{feature}_{i}'] = df_long[feature].shift(i)
        # Append the generated lag column names to the lag_columns list
        lag_columns.append(f'{feature}_{i}')

# Now, lag_columns contains the names of all generated lag columns
df_long['macd_direction'] = np.where(df_long.macdhist > 0, 1, -1)


In [3]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping

# Assuming you have already loaded your data

# Step 1: Select features and target variable
features = lag_columns
target = 'label'

# Encode the labels using LabelEncoder
label_encoder = LabelEncoder()
train_df = df_long[df_long.trade == 1.0]
train_df['label'] = label_encoder.fit_transform(train_df['label'])

# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    train_df[features], train_df[target], test_size=0.2, random_state=42
)

# Step 3: Scale the features (MinMaxScaler)
scaler = MinMaxScaler()
X_train_scaled, X_test_scaled = scaler.fit_transform(X_train), scaler.transform(X_test)

# Count the number of samples in each class
print("Class distribution before oversampling:", Counter(y_train))

# Apply SMOTE (Synthetic Minority Over-sampling Technique)
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Count the number of samples in each class after oversampling
print("Class distribution after oversampling:", Counter(y_train_resampled))

# Step 4: Define the neural network model
model = Sequential([
    Dense(120, input_dim=len(features), activation='relu'),
    Dense(60, activation='relu'),
    Dense(5, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification, so use 'sigmoid' activation
])

# Specify a higher learning rate for the Adam optimizer
learning_rate = 0.01  # You can adjust this value
optimizer = Adam(learning_rate=learning_rate)

# Compile the model with the specified optimizer
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Step 5: Train the model with TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

tensorboard = TensorBoard(log_dir='./logs', histogram_freq=1, write_graph=True, write_images=True)
model.fit(X_train_resampled, y_train_resampled, epochs=5000, batch_size=32, validation_data=(X_test_scaled, y_test), callbacks=[])

# Step 6: Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

# Inverse transform the encoded labels to get the original labels
# original_labels = label_encoder.inverse_transform(model.predict(X_test_scaled) > 0.5)


2023-09-10 04:34:07.305473: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 04:34:08.030769: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-10 04:34:08.032814: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-10 04:34:09.397732: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/tmp/ipykernel_1212224/2018038245.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing

Class distribution before oversampling: Counter({0: 31886, 1: 11251})
Class distribution after oversampling: Counter({0: 31886, 1: 31886})
Epoch 1/5000
1993/1993 [==============================] - 4s 1ms/step - loss: 0.6847 - accuracy: 0.5464 - val_loss: 0.6919 - val_accuracy: 0.4564
Epoch 2/5000
1993/1993 [==============================] - 3s 1ms/step - loss: 0.6830 - accuracy: 0.5533 - val_loss: 0.7149 - val_accuracy: 0.4104
Epoch 3/5000
1993/1993 [==============================] - 3s 1ms/step - loss: 0.6824 - accuracy: 0.5548 - val_loss: 0.6887 - val_accuracy: 0.4775
Epoch 4/5000
1993/1993 [==============================] - 3s 1ms/step - loss: 0.6819 - accuracy: 0.5548 - val_loss: 0.6562 - val_accuracy: 0.5662
Epoch 5/5000
1993/1993 [==============================] - 3s 1ms/step - loss: 0.6815 - accuracy: 0.5556 - val_loss: 0.7072 - val_accuracy: 0.4156
Epoch 6/5000
1993/1993 [==============================] - 3s 1ms/step - loss: 0.6810 - accuracy: 0.5588 - val_loss: 0.6679 - val_ac

KeyboardInterrupt: 

In [86]:
predictions = model.predict(X_test_scaled)
print(predictions.shape)


338/338 [==============================] - 0s 1ms/step
(10785, 1)
